In [1]:
import cv2
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import zipfile

cv2.__version__

'4.10.0'

In [2]:
import tensorflow
tensorflow.__version__

2024-11-15 23:49:41.523945: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 23:49:41.534183: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 23:49:41.545860: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 23:49:41.549487: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 23:49:41.557964: I tensorflow/core/platform/cpu_feature_guar

'2.17.0'

In [3]:
from tensorflow.keras.models import load_model

diretorio = './modelo_01_expressoes.keras'
model = load_model(diretorio)

I0000 00:00:1731725382.754912   40537 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-15 23:49:42.789532: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
arquivo_video = "./Material/Videos/video_teste04.mp4"
cap = cv2.VideoCapture(arquivo_video)

conectado, video = cap.read()
print(video.shape) 

(360, 640, 3)


In [5]:
redimensionar = True
largura_maxima = 600  

if (redimensionar and video.shape[1]>largura_maxima):
  proporcao = video.shape[1] / video.shape[0]
  video_largura = largura_maxima
  video_altura = int(video_largura / proporcao)
else:
  video_largura = video.shape[1]
  video_altura = video.shape[0]

In [6]:
# nome do arquivo de vídeo que será salvo
nome_arquivo = diretorio+'resultado_video_teste04.avi'

# definição do codec
fourcc = cv2.VideoWriter_fourcc(*'XVID')
# FourCC é um código de 4 bytes usado para especificar o codec de vídeo. A lista de códigos disponíveis pode ser encontrada no site fourcc.org
# Codecs mais usados: XVID, MP4V, MJPG, DIVX, X264...
# Por exemplo, para salvar em formato mp4 utiliza-se o codec mp4v (o nome do arquivo também precisa possuir a extensão .mp4)
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# FPS - frames por segundo
fps = 24
# se quiser deixar o video um pouco mais lento pode diminuir o numero de frames por segundo para 20

saida_video = cv2.VideoWriter(nome_arquivo, fourcc, fps, (video_largura, video_altura))

In [12]:
from tensorflow.keras.preprocessing.image import img_to_array
from IPython.display import display

haarcascade_faces = './Material/haarcascade_frontalface_default.xml' # arquivo haarcascade

# define os tamanhos para as fontes
fonte_pequena, fonte_media = 0.4, 0.7

fonte = cv2.FONT_HERSHEY_SIMPLEX

expressoes = ["Raiva", "Nojo", "Medo", "Feliz", "Triste", "Surpreso", "Neutro"]

while (cv2.waitKey(1) < 0):
    conectado, frame = cap.read()

    if not conectado:
        break  # se ocorreu um problema ao carregar a imagem então interrompe o programa

    t = time.time() # tempo atual, antes de iniciar (vamos utilizar para calcular quanto tempo levou para executar as operações)

    # frame_video = np.copy(frame) # faz uma copia do frame do video

    if redimensionar: # se redimensionar = True então redimensiona o frame para os novos tamanhos
      frame = cv2.resize(frame, (video_largura, video_altura))

    face_cascade = cv2.CascadeClassifier(haarcascade_faces)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # converte pra grayscale
    faces = face_cascade.detectMultiScale(gray,scaleFactor=1.2, minNeighbors=5,minSize=(30,30))

    if len(faces) > 0:
        for (x, y, w, h) in faces:

            frame = cv2.rectangle(frame,(x,y),(x+w,y+h+10),(255,50,50),2) # desenha retângulo ao redor da face

            roi = gray[y:y + h, x:x + w]      # extrai apenas a região de interesse (ROI) que é onde contém o rosto
            roi = cv2.resize(roi, (48, 48))    # antes de passar pra rede neural redimensiona para o tamanho das imagens de treinamento
            roi = roi.astype("float") / 255.0  # normaliza
            roi = img_to_array(roi)            # converte para array para que a rede possa processar
            roi = np.expand_dims(roi, axis=0)  # muda o shape da array

            # faz a predição - calcula as probabilidades
            result = model.predict(roi)[0]
            print(result)

            if result is not None:
                resultado = np.argmax(result) # encontra a emoção com maior probabilidade
                cv2.putText(frame,expressoes[resultado],(x,y-10), fonte, fonte_media,(255,255,255),1,cv2.LINE_AA) # escreve a emoção acima do rosto

    # tempo processado = tempo atual (time.time()) - tempo inicial (t)
    cv2.putText(frame, " frame processado em {:.2f} segundos".format(time.time() - t), (20, video_altura-20), fonte, fonte_pequena, (250, 250, 250), 0, lineType=cv2.LINE_AA)

    saida_video.write(frame) # grava o frame atual

print("Terminou")
saida_video.release()
cv2.destroyAllWindows()

Terminou
